In [ ]:
import torch
from torch.nn import Linear, ReLU, Softmax
from torch.utils.data import Dataset, DataLoader
from load_data import load_normalized_data
from torch.nn.functional import relu, softmax
from tqdm import tqdm_notebook, tqdm
from IPython.display import clear_output

X_train, X_test, y_train, y_test, num_classes, class_weights = load_normalized_data("data/denver_data.csv", "weather_description_Denver")
                                                                                    

class NeuralNet(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(NeuralNet, self).__init__()
        self.input_layer = Linear(input_dim, 50)
        self.hidden_layer1 = Linear(50, 100)                                                                             
        self.hidden_layer2 = Linear(100, 200)                                                                             
        self.output_layer = Linear(200, output_dim)   
        # ReLu activation, max of 0 and x, automatic differentiation                                                                             
        self.relu = ReLU()   
                                                                                    
    """
    [input_dim x 50] -> [50 x 100] -> [100 x 200] -> [200 x output_dim]
    """                                                                             
    def forward(self, input_data):  
        input_layer_output = self.relu(self.input_layer(input_data))                                                                              
        hidden_layer_output1 = self.relu(self.hidden_layer1(input_layer_output))                                                                             
        hidden_layer_output2 = self.relu(self.hidden_layer2(hidden_layer_output1))
        output_layer_output = self.output_layer(hidden_layer_output2)                                                                     
        return output_layer_output

class DenverDataset(Dataset):
    """Denver Hourly Weather Dataset."""

    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __len__(self):
        return self.X_data.shape[0]

    def __getitem__(self, idx):
        return self.X_data[idx], self.y_data[idx][0]                                                  
def train(X_train, X_test, y_train, y_test, num_classes, class_weights, epochs, batchSize):

    X_train = torch.tensor(X_train, requires_grad=True, dtype=torch.float)
    y_train = torch.tensor(y_train, dtype=torch.long)
    X_test = torch.tensor(X_test, requires_grad=True, dtype=torch.float)
    y_test = torch.tensor(y_test, dtype=torch.long) 
    
    training_set = DataLoader(DenverDataset(X_train, y_train), batch_size=batchSize, shuffle=True)
    # Define Hyperparams 	                                     

    # Initalize neural network
    model = NeuralNet(X_train.shape[1], num_classes)
    # Calculating the entropy loss using logSoftmax and ngeative log likelihood loss
    criterion = torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float))

    #tqdm -> loading progress bar                                 
    loop = tqdm_notebook(range(epochs))

    for epoch in tqdm_notebook(range(epochs)):
        running_loss = 0.0

        for i, data in enumerate(training_set):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()                      
train(X_train, X_test, y_train, y_test, num_classes, class_weights, 100, 400)



/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
